#Import

In [2]:
#Run if you want to autoreload your personal modules on change
import autoreload
%load_ext autoreload
%autoreload 2



In [6]:
import sys
sys.path

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/ins07202222/code',
 '/anaconda/envs/azureml_py38/lib/python38.zip',
 '/anaconda/envs/azureml_py38/lib/python3.8',
 '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload',
 '',
 '/anaconda/envs/azureml_py38/lib/python3.8/site-packages',
 '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/IPython/extensions',
 '/home/azureuser/.ipython',
 '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor']

In [16]:
sys.path

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/ins07202222/code',
 '/anaconda/envs/azureml_py38/lib/python38.zip',
 '/anaconda/envs/azureml_py38/lib/python3.8',
 '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload',
 '',
 '/anaconda/envs/azureml_py38/lib/python3.8/site-packages',
 '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/IPython/extensions',
 '/home/azureuser/.ipython',
 '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor']

In [ ]:
#Next is needed in azure vm to autoreload modules in cwd
import os
pwd=os.popen("pwd").read().rstrip()

import sys
sys.path.append(pwd)
print("yo")

In [14]:
!cd Users/S242660/

/bin/bash: line 0: cd: Users/Users/S242660/: No such file or directory


In [15]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ins07202222/code


In [3]:
import pytorchfi_c

ModuleNotFoundError: No module named 'pytorchfi_c'

In [ ]:
import py

In [ ]:
%cd drive/MyDrive/Colab\ Notebooks
!ls

/content/drive/MyDrive/Colab Notebooks
 checkpoint	        manyInjections.py
 checkpoint_default     manyInj.py
 data		        MLPR
 decimal_test.csv       models
 decimal_test.npy       __pycache__
 faultInjection.ipynb   results
 load18_default.py      snippets.ipynb
 load18.py	       'Support Vector Machine From Scratch.ipynb'
 main18_default.ipynb   utils_nb.ipynb
 main18.ipynb	        utils.py
 main_default.py        wandb
 main.py


In [ ]:
#Show current devices (GPU)
#print(torch.cuda.current_device())
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2032632446091393088, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14509932544
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14480359891275526803
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [5]:
import sys
sys.path

['/anaconda/envs/azureml_py36/lib/python36.zip',
 '/anaconda/envs/azureml_py36/lib/python3.6',
 '/anaconda/envs/azureml_py36/lib/python3.6/lib-dynload',
 '',
 '/anaconda/envs/azureml_py36/lib/python3.6/site-packages',
 '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost-0.90-py3.6.egg',
 '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/extensions',
 '/home/azureuser/.ipython',
 '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/_project/vendor',
 '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/_vendor']

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar

In [ ]:
!pip install pytorchfi

In [ ]:
from pytorchfi.core import fault_injection as pfi_core
from pytorchfi.errormodels import (
    random_inj_per_layer,
    random_inj_per_layer_batched,
    random_neuron_inj,
    random_neuron_inj_batched,
    random_neuron_single_bit_inj,
    random_neuron_single_bit_inj_batched,
    random_weight_inj
)

import manyInj #Custom fault injector model
import pytorchfi
import numpy

#Run 18

In [ ]:
%run load18.py

Files already downloaded and verified


In [ ]:
test()

 [================================================================>]  Step: 36ms | Tot: 3s770ms | Loss: 0.179 | Acc: 95.520% (9552/10000) 100/100 


In [ ]:
#Function to test a given model
def testM(model):
    global best_acc
    #model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'%(test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        return correct/total


In [ ]:
#Function to test a model with mixed presision (amp). however not giving big time improvements, at least in testing
def testMamp(model):
    global best_acc
    #model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            with torch.cuda.amp.autocast():
              inputs, targets = inputs.to(device), targets.to(device)
              outputs = model(inputs)
              loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'%(test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        return correct/total

In [ ]:
test()

 [================================================================>]  Step: 30ms | Tot: 3s969ms | Loss: 0.179 | Acc: 95.520% (9552/10000) 100/100 


In [ ]:
testMamp(net)

Loss: 0.179 | Acc: 95.520% (9552/10000)


0.9552

In [ ]:
#Change test batch size
num_batches=100
test_batch_size=int(numpy.shape(testset.data)[0]/num_batches)
#test_batch_size=100
testloader = torch.utils.data.DataLoader(
    testset, batch_size=test_batch_size, shuffle=False, num_workers=2)


In [ ]:
original_acc=testM(net)

Loss: 0.179 | Acc: 95.520% (9552/10000)


In [ ]:
test_features, test_labels = next(iter(testloader))
#Get num channels, width and height of input data
c,w,h=list(test_features[0].size())
#Batchsize, see load18.py or main.py for batch size
num_batches=int(numpy.shape(testset.data)[0]/test_batch_size)

In [ ]:
#We introduce the test batch size because we may select the batch element on which apply an injection
pfi_model = pfi_core(net, h, w, test_batch_size, c=c,debug=True,use_cuda=True)

In [ ]:
testM(pfi_model.ORIG_MODEL)

Loss: 0.179 | Acc: 95.520% (9552/10000)


0.9552

In [ ]:
#Get number of neurons per layer
outputs=pfi_model.OUTPUT_SIZE
total_neurons=sum(numpy.prod(outputs,axis=1))
n_per_layer=numpy.prod(outputs,axis=1)

In [ ]:
#test injection for 1 layer
import time
l=17
inj_perc=0.4
start = time.process_time()
print("layer: ",l)
n_inj=int(n_per_layer[l]*inj_perc)
print("n_inj: ",n_inj)
many_layer=manyInj.many_n_inj_layer(pfi_model=pfi_model,n_inj=n_inj,layer=l,min_val=0,max_val=0)
print("testing")
inj_acc=testMamp(many_layer)
print("testing Orig")
orig_acc=testMamp(pfi_model.ORIG_MODEL)
print("Diff: ",orig_acc-inj_acc)
print("Time in layer: ",time.process_time() - start)

layer:  17
n_inj:  3276
N injections:  3276
testing
Loss: 0.179 | Acc: 95.520% (9552/10000)
testing Orig
Loss: 0.179 | Acc: 95.520% (9552/10000)
Diff:  0.0
Time in layer:  165.57790371


In [ ]:
#Do injections for every layer with a percentage of layer neurons
import time
inj_perc=0.4
print("Injection percentage for each layer: " ,inj_perc)
print("test batch size: ", test_batch_size)
print("test batch number: ", num_batches)
inj_acc_v=[]
orig_acc_v=[]
times_v=[]
for l in range(pfi_model.get_total_conv()):
  start = time.process_time()
  print("layer: ",l)
  n_inj=int(n_per_layer[l]*inj_perc)
  print("n_inj: ",n_inj)
  many_layer=manyInj.many_n_inj_layer(pfi_model=pfi_model,n_inj=n_inj,layer=l,min_val=0,max_val=0)
  print("testing")
  inj_acc=testM(many_layer)
  print("testing Orig")
  orig_acc=testM(pfi_model.ORIG_MODEL)
  print("Diff: ",orig_acc-inj_acc)
  inj_acc_v.append(inj_acc)
  orig_acc_v.append(original_acc)
  times_v.append(time.process_time() - start)
  print("Time in layer: ",time.process_time() - start)

Injection percentage for each layer:  0.4
test batch size:  100
test batch number:  100
layer:  0
n_inj:  26214
N injections:  26214
testing
Loss: 0.180 | Acc: 95.510% (9551/10000)
testing Orig
Loss: 0.179 | Acc: 95.520% (9552/10000)
Diff:  0.00010000000000010001
Time in layer:  1441.761342555
layer:  1
n_inj:  26214
N injections:  26214
testing
Loss: 0.180 | Acc: 95.480% (9548/10000)
testing Orig
Loss: 0.179 | Acc: 95.520% (9552/10000)
Diff:  0.00040000000000006697
Time in layer:  1444.881089298
layer:  2
n_inj:  26214
N injections:  26214
testing
Loss: 0.179 | Acc: 95.520% (9552/10000)
testing Orig
Loss: 0.179 | Acc: 95.520% (9552/10000)
Diff:  0.0
Time in layer:  1438.5011066560005
layer:  3
n_inj:  26214
N injections:  26214
testing
Loss: 0.179 | Acc: 95.510% (9551/10000)
testing Orig
Loss: 0.179 | Acc: 95.520% (9552/10000)
Diff:  0.00010000000000010001
Time in layer:  1439.4414176700002
layer:  4
n_inj:  26214
N injections:  26214
testing
Loss: 0.179 | Acc: 95.510% (9551/10000)
te

In [ ]:
!mkdir -p results/40

In [ ]:
#Save results
import json
with open("./results/40/"+"inj_acc_v.json", 'w') as fh:
  json.dump(inj_acc_v,fh)
with open("results/40/"+"orig_acc_v.json", 'w') as fh:
  json.dump(orig_acc_v,fh)
with open("results/40/"+"times_v.json", 'w') as fh:
  json.dump(times_v,fh)

NameError: ignored

In [ ]:
import json
with open("./results/20/inj_acc_v.json", 'r') as fh:
  inj_acc=json.load(fh)
with open("results/20/orig_acc_v.json", 'r') as fh:
  orig_acc=json.load(fh)

In [ ]:
import numpy
numpy.mean(numpy.array(orig_acc) - numpy.array(inj_acc))

1.5000000000037205e-05

#Other tests

In [ ]:

import decimal
from numpy import asarray
from numpy import savetxt
from numpy import save
# define data
data = asarray([[decimal.Decimal(1),[decimal.Decimal(1),decimal.Decimal(1)]],[decimal.Decimal(1),[decimal.Decimal(1),decimal.Decimal(1)]]])
# save to csv file
save('decimal_test.npy', data)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
# load numpy array from npy file
from numpy import load
# load array
data = load('decimal_test.npy',allow_pickle=True)
# print the array
data

array([[Decimal('1'), list([Decimal('1'), Decimal('1')])],
       [Decimal('1'), list([Decimal('1'), Decimal('1')])]], dtype=object)

In [ ]:
data[0][1]

[Decimal('1'), Decimal('1')]

In [ ]:

many_layer=manyInj.many_n_inj_layer(pfi_model=pfi_model,n_inj=n_inj,layer=layer,min_val=0,max_val=0)

N injections:  13107


In [ ]:
len(pfi_model.CORRUPT_CONV)

1310700

In [ ]:
inj_acc=testM(many_layer)

Loss: 0.238 | Acc: 93.570% (9357/10000)


1000

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0) 
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

In [ ]:
torch.cuda.empty_cache()

In [ ]:
test_batch_size

1000

In [ ]:
rnib=random_neuron_inj_batched(pfi_model)
rnpl=random_inj_per_layer_batched(pfi_model)

In [ ]:
testM(rnib)

Loss: 0.179 | Acc: 95.520% (9552/10000)


In [ ]:
testM(net)

Loss: 0.179 | Acc: 95.520% (9552/10000)


In [ ]:
testM(rnpl)

Loss: 0.179 | Acc: 95.530% (9553/10000)


In [ ]:
many_model = many_n_injections(pfi_model,1000,min_val=0,max_val=0)

N injections:  1000


In [ ]:
testM(many_model)

Loss: 0.563 | Acc: 81.040% (8104/10000)


In [ ]:
#Number of neurons as for pytFI

np.prod(pfi_model.OUTPUT_SIZE)
accum=0
for l in pfi_model.OUTPUT_SIZE:
  accum += np.prod(l)
  print(np.prod(l))
accum

16384
4096
4096
4096
4096
2048
2048
2048
2048
2048
1024
1024
1024
1024
1024
512
512
512
512
512


50688

In [ ]:
j=0
for name, param in pfi_model.get_original_model().named_parameters():
    if "conv" in name or "downsample.0" in name: 
      j=j+1
      print(name,"|",param.size())


module.conv1.weight | torch.Size([64, 3, 7, 7])
module.layer1.0.conv1.weight | torch.Size([64, 64, 3, 3])
module.layer1.0.conv2.weight | torch.Size([64, 64, 3, 3])
module.layer1.1.conv1.weight | torch.Size([64, 64, 3, 3])
module.layer1.1.conv2.weight | torch.Size([64, 64, 3, 3])
module.layer2.0.conv1.weight | torch.Size([128, 64, 3, 3])
module.layer2.0.conv2.weight | torch.Size([128, 128, 3, 3])
module.layer2.0.downsample.0.weight | torch.Size([128, 64, 1, 1])
module.layer2.1.conv1.weight | torch.Size([128, 128, 3, 3])
module.layer2.1.conv2.weight | torch.Size([128, 128, 3, 3])
module.layer3.0.conv1.weight | torch.Size([256, 128, 3, 3])
module.layer3.0.conv2.weight | torch.Size([256, 256, 3, 3])
module.layer3.0.downsample.0.weight | torch.Size([256, 128, 1, 1])
module.layer3.1.conv1.weight | torch.Size([256, 256, 3, 3])
module.layer3.1.conv2.weight | torch.Size([256, 256, 3, 3])
module.layer4.0.conv1.weight | torch.Size([512, 256, 3, 3])
module.layer4.0.conv2.weight | torch.Size([512, 

In [ ]:
list(pfi_model.ORIG_MODEL.modules())

NameError: ignored

In [ ]:
pfi_model.get_original_model()

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [ ]:
list(pfi_model.get_original_model().named_parameters())[0]

AttributeError: ignored

In [ ]:
random_w_model=random_weight_inj(pfi_model,10)

AssertionError: ignored

In [ ]:
many_w_model=many_w_injections(pfi_model,100)

N injections:  100


AssertionError: ignored

In [ ]:
import pytorchfi
print(pytorchfi.__file__)

ModuleNotFoundError: ignored

From here I'll try to load trained weights to default pytorch resnet18

In [ ]:
%run load18_default.py

Files already downloaded and verified


In [ ]:
test()

 [================================================================>]  Step: 63ms | Tot: 3s43ms | Loss: 0.622 | Acc: 79.530% (7953/10000) 100/100 
